# This notebook visualize the reserve margin metric at different temporal resolutions and scenario attributes

# Find vulnerable hourly at the daily resolution

In [3]:
import os
import pandas as pd
import plotly.express as px

In [4]:
def RM_daily(folder_path, policyYear, RMValue):
    # List to store data from each CSV file
    data_list = []

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.startswith(str(policyYear)) and file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Read the CSV file
            df = pd.read_csv(file_path)
            # Add a column to indicate the policy (extracted from the file name)
            df['Policy'] = file_name.split('.')[0].split('_P')[0]
            # Convert datetime column to datetime type
            df['datetime'] = pd.to_datetime(df['datetime'])
            # Extract day of the year from datetime
            df['day_of_year'] = df['datetime'].dt.dayofyear
            # Count cases where PRM falls below 17 for each day of the year
            df['below_17'] = df['PRM'] < RMValue
            df_count = df.groupby(['Policy', 'day_of_year'])['below_17'].sum().reset_index()
            data_list.append(df_count)

    # Concatenate all dataframes
    combined_df = pd.concat(data_list)

    return combined_df

def create_heatmap(combined_df, RMValue, policyYear):
    policy_order = [f'{policyYear}-LeastCost_25MMT', f'{policyYear}-Core_25MMT', f'{policyYear}-High_Gas_Ret']

    # Reorder the DataFrame based on the policy order
    combined_df['Policy'] = pd.Categorical(combined_df['Policy'], categories=policy_order, ordered=True)
    combined_df = combined_df.sort_values('Policy')

    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = combined_df.pivot(index='Policy', columns='day_of_year', values='below_17')

    # Create a heatmap with a white to red color scale
    fig = px.imshow(heatmap_data, labels=dict(x="Day of Year", y="Policy", color="Cases of Vulnerable Hours"),
                    title=f"Number of Cases Below {RMValue} PRM for Each Policy {policyYear}",
                    color_continuous_scale='Reds')

    # Update x-axis labels to show the beginning of each month
    month_start_days = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
    month_labels = ['01/01/2045', '02/01/2045', '03/01/2045', '04/01/2045', '05/01/2045', '06/01/2045',
                    '07/01/2045', '08/01/2045', '09/01/2045', '10/01/2045', '11/01/2045', '12/01/2045']
    fig.update_xaxes(tickvals=month_start_days, ticktext=month_labels)

    # Show the plot
    fig.show()

In [5]:
# Example usage
folder_path = '/Users/liyangwang/kitxDMDU_analysis/reservemargin'
policyYear = 2045
RMValue = 5
combined_df = RM_daily(folder_path, policyYear, RMValue)
create_heatmap(combined_df, RMValue, policyYear)

# Find the reserve margin at the hourly level for specific month

In [6]:

def RM_hourly(folder_path, policyYear, month, RMValue):
    # List to store data from each CSV file
    data_list = []

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.startswith(str(policyYear)) and file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Read the CSV file
            df = pd.read_csv(file_path)
            # Add a column to indicate the policy (extracted from the file name)
            df['Policy'] = file_name.split('.')[0].split('_P')[0]
            # Convert datetime column to datetime type
            df['datetime'] = pd.to_datetime(df['datetime'])
            # Filter for the specified month and create a copy
            df_filtered = df.loc[(df['datetime'].dt.year == policyYear) & (df['datetime'].dt.month == month)].copy()
            # Extract hour from datetime
            df_filtered['hour'] = df_filtered['datetime'].dt.hour
            # Count cases where PRM falls below 17 for each hour
            df_filtered['below_17'] = df_filtered['PRM'] < RMValue
            df_count = df_filtered.groupby(['Policy', 'hour'])['below_17'].sum().reset_index()
            data_list.append(df_count)

    # Concatenate all dataframes
    combined_df = pd.concat(data_list)

    return combined_df

def create_heatmap(combined_df, policyYear):
    # Define the order of policies dynamically based on the policyYear
    policy_order = [f'{policyYear}-LeastCost_25MMT', f'{policyYear}-Core_25MMT', f'{policyYear}-High_Gas_Ret']

    # Reorder the DataFrame based on the policy order
    combined_df['Policy'] = pd.Categorical(combined_df['Policy'], categories=policy_order, ordered=True)
    combined_df = combined_df.sort_values('Policy')

    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = combined_df.pivot(index='Policy', columns='hour', values='below_17')

    # Create a heatmap with a white to red color scale
    fig = px.imshow(heatmap_data, labels=dict(x="Hour", y="Policy", color="Cases of Vulnerable Hours"),
                    title=f"Number of Cases of Vulnerable Hours for Each Policy in {policyYear}",
                    color_continuous_scale='Reds')

    # Show the plot
    fig.show()


In [7]:
# Specify the folder path where the CSV files are stored along with other inputs for the function 
folder_path = '/Users/liyangwang/kitxDMDU_analysis/reservemargin'
policyYear = 2045
month = 8  # Specify the month (e.g., 9 for September)
RMValue = 5  # Specify the reserve margin value

combined_df = RM_hourly(folder_path, policyYear, month, RMValue)
create_heatmap(combined_df, policyYear)

In [8]:
def hourly_ClimateModel_SSP(folder_path, month, RMValue):
    # List to store data from each CSV file
    data_list = []

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.startswith('2045') and file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Read the CSV file
            df = pd.read_csv(file_path)
            # Add a column to indicate the policy (extracted from the file name)
            df['Policy'] = file_name.split('.')[0].split('_P')[0]
            # Convert datetime column to datetime type
            df['datetime'] = pd.to_datetime(df['datetime'])
            # Extract SSP and climate model from the case column
            df['SSP'] = df['case'].apply(lambda x: x.split('_')[1])
            df['Climate Model'] = df['case'].apply(lambda x: x.split('_')[2])
            # Filter for September 2045 and create a copy
            df_september = df.loc[(df['datetime'].dt.year == 2045) & (df['datetime'].dt.month == month)].copy()
            # Extract hour from datetime
            df_september['hour'] = df_september['datetime'].dt.hour
            # Count cases where PRM falls below 17 for each hour
            df_september['below_17'] = df_september['PRM'] < RMValue
            df_count = df_september.groupby(['Policy', 'SSP', 'Climate Model', 'hour'])['below_17'].sum().reset_index()
            data_list.append(df_count)

    # Concatenate all dataframes
    combined_df = pd.concat(data_list)

    return combined_df

def create_heatmap(combined_df, RMValue):
    # Define the order of policies
    policy_order = [f'{policyYear}-LeastCost_25MMT', f'{policyYear}-Core_25MMT', f'{policyYear}-High_Gas_Ret']

    # Reorder the DataFrame based on the policy order
    combined_df['Policy'] = pd.Categorical(combined_df['Policy'], categories=policy_order, ordered=True)
    combined_df = combined_df.sort_values('Policy')

    # Convert Categorical to string before concatenating
    combined_df['Policy'] = combined_df['Policy'].astype(str)
    combined_df['Policy_SSP_Climate_Model'] = combined_df['Policy'] + '-' + combined_df['SSP'] + '-' + combined_df['Climate Model']

    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = combined_df.pivot(index='Policy_SSP_Climate_Model', columns='hour', values='below_17')

    # Create a heatmap with a white to red color scale
    fig = px.imshow(heatmap_data, labels=dict(x="Hour", y="Policy, SSP, and Climate Model", color="Cases of Vulnerable Hours"),
                    title = "Number of Cases Below {} Reserve Margin for Each Policy, SSP, and Climate Model in September 2045".format(RMValue),
                    color_continuous_scale='Reds')

    # Update layout to make the figure bigger
    fig.update_layout(width=1200, height=800)

    # Show the plot
    fig.show()

In [9]:
folder_path = '/Users/liyangwang/kitxDMDU_analysis/reservemargin'
RMValue = 10
combined_df = hourly_ClimateModel_SSP(folder_path, 9, RMValue)
create_heatmap(combined_df, RMValue)

#examine the impact of supply via weather year

In [10]:
import os
import pandas as pd
import plotly.express as px

def RM_weatherYear(folder_path, month):
    # List to store data from each CSV file
    data_list = []

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.startswith('2045') and file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Read the CSV file
            df = pd.read_csv(file_path)
            # Add a column to indicate the policy (extracted from the file name)
            df['Policy'] = file_name.split('.')[0].split('_P')[0]
            # Convert datetime column to datetime type
            df['datetime'] = pd.to_datetime(df['datetime'])
            # Extract weather year from the case column (section after the last '_')
            df['Weather Year'] = df['case'].apply(lambda x: x.split('_')[-1])
            # Filter for the specified month and create a copy
            df_filtered = df.loc[(df['datetime'].dt.year == 2045) & (df['datetime'].dt.month == month)].copy()
            # Extract hour from datetime
            df_filtered['hour'] = df_filtered['datetime'].dt.hour
            # Count cases where PRM falls below 17 for each hour
            df_filtered['below_17'] = df_filtered['PRM'] < 17
            df_count = df_filtered.groupby(['Policy', 'Weather Year', 'hour'])['below_17'].sum().reset_index()
            data_list.append(df_count)

    # Concatenate all dataframes
    combined_df = pd.concat(data_list)

    return combined_df

def create_heatmap(combined_df, month, RMValue):
    # Define the order of policies
    policy_order = [f'{policyYear}-LeastCost_25MMT', f'{policyYear}-Core_25MMT', f'{policyYear}-High_Gas_Ret']

    # Reorder the DataFrame based on the policy order
    combined_df['Policy'] = pd.Categorical(combined_df['Policy'], categories=policy_order, ordered=True)
    combined_df = combined_df.sort_values('Policy')

    # Convert Categorical to string before concatenating
    combined_df['Policy'] = combined_df['Policy'].astype(str)
    combined_df['Policy_Weather_Year'] = combined_df['Policy'] + '-' + combined_df['Weather Year']

    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = combined_df.pivot(index='Policy_Weather_Year', columns='hour', values='below_17')

    # Create a heatmap with a white to red color scale
    fig = px.imshow(heatmap_data, labels=dict(x="Hour", y="Policy and Weather Year", color="Cases of Vulnerable Hours"),
                    title=f"Number of Cases Below {RMValue} PRM for Each Policy and Weather Year in {month:02d}/2045",
                    color_continuous_scale='Reds')

    # Update layout to make the figure bigger
    fig.update_layout(width=1200, height=800)

    # Show the plot
    fig.show()

# Example usage
month = 9  # Specify the month (e.g., 9 for September)
RMValue = 5
combined_df = RM_weatherYear(folder_path, month)
create_heatmap(combined_df, month, RMValue)

In [11]:
import os
import pandas as pd
import plotly.express as px

def RM_DailyClimateModel(folder_path, policyYear):
    # List to store data from each CSV file
    data_list = []

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.startswith(policyYear) and file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            # Read the CSV file
            df = pd.read_csv(file_path)
            # Add a column to indicate the policy (extracted from the file name)
            df['Policy'] = file_name.split('.')[0].split('_P')[0]
            # Convert datetime column to datetime type
            df['datetime'] = pd.to_datetime(df['datetime'])
            # Extract day of the year from datetime
            df['day_of_year'] = df['datetime'].dt.dayofyear
            # Extract SSP and climate model from the case column
            df['SSP'] = df['case'].apply(lambda x: x.split('_')[1])
            df['Climate Model'] = df['case'].apply(lambda x: x.split('_')[2])
            # Filter for SSP 3_7
            df = df[df['SSP'] == '7']
            # Count cases where PRM falls below 17 for each day of the year
            df['below_17'] = df['PRM'] < 17
            df_count = df.groupby(['Policy', 'Climate Model', 'day_of_year'])['below_17'].sum().reset_index()
            data_list.append(df_count)

    # Concatenate all dataframes
    combined_df = pd.concat(data_list)

    return combined_df

def create_heatmap(combined_df, RMValue, policyYear):
    # Concatenate Policy and Climate Model into a single string
    combined_df['Policy_Climate_Model'] = combined_df['Policy'] + ' - ' + combined_df['Climate Model']

    # Pivot the dataframe to create a matrix for the heatmap
    heatmap_data = combined_df.pivot(index='Policy_Climate_Model', columns='day_of_year', values='below_17')

    # Create a heatmap with a white to red color scale
    fig = px.imshow(heatmap_data, labels=dict(x="Day of Year", y="Policy and Climate Model", color="Cases of Vulnerable Hours"),
                    title=f"Number of Cases Below {RMValue} Reserve Margin for Each Policy and Climate Model in {policyYear}",
                    color_continuous_scale='Reds')

    # Update x-axis labels to show the beginning of each month
    month_start_days = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
    month_labels = ['01/01/2045', '02/01/2045', '03/01/2045', '04/01/2045', '05/01/2045', '06/01/2045',
                    '07/01/2045', '08/01/2045', '09/01/2045', '10/01/2045', '11/01/2045', '12/01/2045']
    fig.update_xaxes(tickvals=month_start_days, ticktext=month_labels)

     # Update layout to make the figure bigger
    fig.update_layout(width=1200, height=800)

    # Show the plot
    fig.show()



In [12]:
combined_df = RM_DailyClimateModel(folder_path, '2045')
create_heatmap(combined_df, RMValue, policyYear)